## Contacts App
> Following [Hypermedia Systems book](https://hypermedia.systems)

In [1]:
#| default_exp contacts

In [2]:
#| export
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
from monsterui.all import *
import json

In [3]:
#| export
app,rt,contacts,Contact = fast_app("data/contacts.db",hdrs=Theme.blue.headers(),
                                    id=int, first=str, last=str, phone=str, email=str, errors=str,
                                    pk="id", live=True)

In [4]:
#| export
if not contacts(): 
    with open("contacts.json","r") as f: data = json.load(f)
    contacts.insert_all(data)

### Table view

In [5]:
#| export
@rt("/")
def get(): return Redirect("/contacts")


@rt("/contacts")
def get(q:str=None):
    search = Form(
        DivHStacked(
            Input(name="q", value=q, placeholder="Search contacts...", cls="w-full md:w-2/3 lg:w-1/2", hx_get="/contacts/search",
                  hx_trigger="keyup changed delay:500ms", hx_target="#contacts-table", hx_include='[name="q"]'),
            Button("Search", type="submit")
       ),
       cls="mt-8"
    )
    return Container(page_heading, search, contacts_table(q), add_button, Div(id="modal-container"), Div(id="success-toast"))

@rt("/contacts/search")
def get(q: str = ''): return contacts_table(q)

### Edit Contacts

In [6]:
@rt("/contacts/{id:int}/edit")
def get(id:int):
    contact=contacts[id]
    
    return Modal(
        contact_form(contact, action=f"/contacts/{id}/update"),
        header=ModalHeader(H3("Edit Contact")),
        id="contact-modal",
        open=True
    )

@rt("/contacts/{id:int}/update")
def post(id: int, contact: Contact):
    return handle_contact_save(contact, id)

### New Contacts

In [7]:
#| export
@rt("/contacts/new")
def get():
    """Create a modal for adding a contact"""
    return Modal(
        contact_form(),
        header=ModalHeader(H3("Add Contact")),
        id="contact-modal",
        open=True
    )

@rt("/dismiss-toast")
def get(): return Div(id="success-toast")  

In [8]:
#| export
#Try routing to /new as in the book
@rt("/contacts/create")
def post(contact: Contact):
    return handle_contact_save(contact)

### Detail view

In [9]:
#| export
@rt("/contacts/{id:int}")
def get(id:int): return contact_detail(contacts[id])

### Helper functions

In [18]:
#| export
def header_render(col):
    """Render header for a table"""
    return Th(col.title() if col != "actions" else "", expand=True)

def action_buttons(contact):
    """Add buttons to view and edit contact"""
    return DivHStacked(
        Button(UkIcon("eye", cls="mr-2"), "View", cls=ButtonT.ghost, hx_get=f'/contacts/{contact.id}', hx_target='#modal-container'),
        Button(UkIcon("pencil", cls="mr-2"),"Edit", cls=ButtonT.ghost, hx_get=f'/contacts/{contact.id}/edit', hx_target='#modal-container')
    )

def contacts_table(q=None):
    """Create a table with contacts applying a filter is query was provided"""
    rows = filter_contacts(q)
    return Table(
        Thead(
            Tr(*[Th(col) for col in ["First", "Last", "Phone", "Email", "Actions"]])
        ),
        Tbody(
            *[Tr(
                Td(c.first or ""),
                Td(c.last or ""),
                Td(c.phone or ""),
                Td(c.email or ""),
                Td(action_buttons(c))
            ) for c in rows]
        ),
        cls=(TableT.hover, TableT.divider, TableT.responsive),
        id="contacts-table"
    )

def filter_contacts(q=None):
    """Filter contacts that have `q` in first name, last name or email"""
    if not q: return contacts(order_by="id")
    q = q.lower()
    return [o for o in contacts(order_by="id")
            if q in (o.first or "").lower()
            or q in (o.last or "").lower()
            or q in (o.email or "").lower()]

In [27]:
#| export
def handle_contact_save(contact, id=None):
    """Common logic for saving a contact (create or update)"""
    # validation for errors (TBD: refactor later)
    errors = {}
    if not contact.first: errors["first"] = "First name is required"
    if not contact.email: errors["email"] = "Email is required"
    elif any(c.email == contact.email and (c.id != id) for c in contacts()): errors["email"] = "Email already exists" 

    if errors: 
        contact.errors = json.dumps(errors)
        action = '/contacts/create' if id is None else f'/contacts/{id}/update'
        return Modal(
            contact_form(contact, action), 
            header=ModalHeader(H3("Edit Contact" if id else "Add Contact")),
            id="contact-modal", open=True)

    # If validation passes, save the contact
    if id:
        # Update existing contact
        contact.id = id
        contacts.update(contact)
        message = "Contact updated successfully!"
    else:
        # Create new contact
        contacts.insert(contact)
        message = "Contact added successfully!"

    # create an out-of-band replacement for the contacts table
    updated_table = Div(contacts_table(),id="contacts-table", hx_swap_oob="true")

    # Create a toast notification that appears in the center of the screen
    success_toast = Toast(
        DivLAligned(UkIcon("check-circle", cls='ml-2 text-success'), Span(message)),
        id="success-toast", alert_cls=AlertT.success, cls=(ToastHT.center, ToastVT.middle), hx_swap_oob="true",
        hx_get="/dismiss-toast", hx_trigger="load delay:3s", hx_target="#success-toast")
    
    # Return both components
    return Div(updated_table, success_toast)

In [28]:
#| export
def contact_form(contact=None, action="/contacts/create"):
    """Reusable form that works both for new and edit"""

    # parse errors if they exist
    errors = json.loads(contact.errors) if contact and contact.errors else {}

    return Form(
        Grid(
            Div(
                LabelInput("First Name", id="first", placeholder="First Name", value=contact.first if contact else ""),
                P(errors.get("first", ""), cls="text-error text-sm mt-1") if "first" in errors else None),
            Div(
                LabelInput("Last Name", id="last", placeholder="Last Name", value=contact.last if contact else "")), cols=2),
        Grid(
            Div(
                LabelInput("Phone", id="phone", placeholder='Phone', value=contact.phone if contact else "")),
            Div(
                LabelInput("Email", id="email", placeholder='Email', value=contact.email if contact else ""),
                P(errors.get("email",""), cls="text-error text-sm mt-1") if "email" in errors else None), cols=2),
        DivRAligned(
            ModalCloseButton("Cancel", cls=ButtonT.ghost),
            Button("Save", cls=ButtonT.primary, type="submit")),
        hx_post=action,
        hx_target="#modal-container"
    )

In [29]:
#| export
def contact_detail(contact):
    """Detail view for a contact"""

    return Modal(
        ModalBody(
            # Use a single centered container for the entire content
            DivCentered(
                # Name at the top, centered
                H4(f"{contact.first or ''} {contact.last or ''}", cls=(TextT.bold, TextT.center, "mb-6")),
                
                # Centered container for phone row
                DivLAligned(UkIcon('phone', cls='mr-2 text-primary'), P(contact.phone or '-'), cls="mb-4 justify-center"),
                
                # Centered container for email row
                DivLAligned(UkIcon('mail', cls='mr-2 text-primary'), P(contact.email or '-'), cls="justify-center"),
                
                # Make sure the outer container allows vertical stacking
                cls="items-center py-4"
            )
        ),
        header = ModalHeader(H3("Contact Details")),
        footer = DivFullySpaced(
            ModalCloseButton("Close", cls=ButtonT.primary),
            Button(UkIcon("pencil", cls="mr-2"), "Edit", cls=ButtonT.primary, hx_get=f"/contacts/{contact.id}/edit", 
                   hx_target="#modal-container")
        ),    
        id = "contact-modal",
        open=True  # This ensures the modal opens automatically
    )

### Components

In [30]:
#| export
page_heading = Div(cls="space-y-2")(H1("Contacts"), P("Manage your contacts!", cls=TextPresets.muted_sm))

add_button = DivLAligned(
    Button(UkIcon("plus-circle", cls="mr-2"), "Add Contact", cls=ButtonT.primary, hx_get="/contacts/new", hx_target="#modal-container"),
    cls="mb-4 mt-4"
)

### Support functions

In [23]:
#contacts.delete_where("id>=20")
# contacts()

In [31]:
server.stop()

In [32]:
server = JupyUvi(app)

In [33]:
HTMX()

In [14]:
# running at http://localhost:8000/

In [ ]:
#| export
serve()

In [35]:
#from nbdev.export import nb_export
nb_export('Contacts_v2.ipynb')